In [3]:
import requests
import os

books = [
    "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%201%20-%20The%20Philosopher's%20Stone.txt",
    # "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%202%20-%20The%20Chamber%20of%20Secrets.txt",
    # "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%203%20-%20The%20Prisoner%20of%20Azkaban.txt",
    # "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%204%20-%20The%20Goblet%20of%20Fire.txt",
    # "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%205%20-%20The%20Order%20of%20the%20Phoenix.txt",
    # "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%206%20-%20The%20Half%20Blood%20Prince.txt",
    # "https://raw.githubusercontent.com/formcept/whiteboard/master/nbviewer/notebooks/data/harrypotter/Book%207%20-%20The%20Deathly%20Hallows.txt"
]

if not os.path.exists('books'):
    os.makedirs('books')

for i, book in enumerate(books):
    with open(f'books/book_{i}.txt', 'w+') as book_file:
        book_file.write(requests.get(book).text)


In [4]:
import re

paragraphs = []

for i, book in enumerate(books):
    with open(f'books/book_{i}.txt', 'r') as book_file:
        text = book_file.read()
        text = re.sub(r'^[Pp]age.*?$', '', text, flags=re.MULTILINE)
        paragraphs.extend(re.split(r'\n\s*\n', text))
        
paragraphs = [p for p in paragraphs if len(p) > 50]

In [24]:
import random
pargraphs = random.shuffle(paragraphs)

In [25]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')

In [27]:
generated_queries = []

for paragraph in paragraphs[:30]:
    input_ids = tokenizer.encode(paragraph, return_tensors='pt')
    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=3)

    generated_queries.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])

In [28]:
from sentence_transformers import SentenceTransformer, util

bi_encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [29]:
encoded_queries = bi_encoder.encode(generated_queries, show_progress_bar=True)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
query = bi_encoder.encode(['who is the headmaster'])
results = util.semantic_search(query, encoded_queries, top_k=5)
for result in results[0]:
    print(generated_queries[result['corpus_id']])

who is the headmaster
is it bad to stay up late with the headmaster
who's that teacher talking to professor quirrell?
who was the professor that looked up professor mcgonagall
who's that teacher talking to professor quirrell
